# Senzing Quickstart with Spark/Databricks

This tutorial is an introduction to using [Senzing](https://senzing.com/) with Spark dataframes. We'll load three example datasets from Spark dataframes into an instance of Senzing, and we'll find all the entities that are present within this data. This will show us duplicates within the data, and we'll gain the capability to merge the dataframes based on the results from Senzing.

We'll use the Senzing "Truthsets" demo data from https://github.com/Senzing/truth-sets/. The datasets are as follows:
- `customers`, a messy dataset of customer names and incomplete PII data. It includes addresses, dates of birth, emails, etc.
- `reference`, containing customer and organization information, with incomplete contact data
- `watchlist`, a list of fraudulent entities

We'll use all three datasets to figure out which rows in the `customers` dataset refer to the same entity (person).

We'll use the [Senzing V4](https://www.senzing.com/docs/4_beta/python/index.html) syntax and a sandbox Senzing gRPC server hosted within a Docker container.

### Steps in this tutorial

TODO what are all the steps

## Set up requirements

In this tutorial, we'll use the [`senzing`](https://garage.senzing.com/sz-sdk-python/index.html) and [`senzing_grpc`](https://garage.senzing.com/sz-sdk-python-grpc/) packages, in addition to PySpark. You can install both of these using `pip`, or use the `requirements.txt` file in the repo folder containing this tutorial.

TODO how to set up gRPC server

Run senzing/serve-grpc with `docker run -it --publish 8261:8261 --rm senzing/serve-grpc`

https://github.com/senzing-garage/serve-grpc/tree/main 

In [1]:
import grpc
from senzing import SzEngineFlags, SzError
from senzing_grpc import SzAbstractFactoryGrpc
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_list, array, array_except
import json

In [ ]:
# TODO download data if it doesn't already exist

## Load data into Spark DataFrames


First, we'll start a new Spark session.

In [2]:
spark = (
    SparkSession.builder.appName("Senzing Quickstart").master("local[*]").getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/08 10:51:37 WARN Utils: Your hostname, Catherines-MacBook-Air-2.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.4 instead (on interface en0)
25/07/08 10:51:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/08 10:51:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 57257)
Traceback (most recent call last):
  File "/Users/catherine/.pyenv/versions/3.13.0/lib/python3.13/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
    ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/catherine/.pyenv/versions/3.13.0/lib/python3.13/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
    ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/catherine/.pyenv/versions/3.13.0/lib/python3.13/socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
    ~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/catherine/.pyenv/versions/3.13.0/lib/python3.13/socketserver.py", line 761, in __init__
    self.handle()
    ~~~~~~~~~~~^^
  File "/Users/catherine/.pyenv/versions/3.13.0/envs/sen

Next, we'll load the three datasets from JSON files into Spark dataframes.

We'll omit the last 10 lines in the `customers` dataset, because we'll use these to show how to add new entries with Spark Streaming in the next tutorial (link).

In [ ]:
customers = spark.read.json("data/customers.json").limit(110)

In [10]:
customers.show(10)

+-------------+------------+---------+--------------------+----------------+----------+---------+------+--------+-----------+-------+-------------+----------------------+---------------------+------------------+------+-------------------+------------------+----------------+----------------+---------------+------------+----------+------------------+-----------------+-----------------+-------------------+----------------+---------+-----------+------------------+-----------+----------+
|    ADDR_CITY|ADDR_COUNTRY|ADDR_FULL|          ADDR_LINE1|ADDR_POSTAL_CODE|ADDR_STATE|ADDR_TYPE|AMOUNT|CATEGORY|DATA_SOURCE|   DATE|DATE_OF_BIRTH|DRIVERS_LICENSE_NUMBER|DRIVERS_LICENSE_STATE|     EMAIL_ADDRESS|GENDER|NATIONAL_ID_COUNTRY|NATIONAL_ID_NUMBER|NATIVE_NAME_FULL|PASSPORT_COUNTRY|PASSPORT_NUMBER|PHONE_NUMBER|PHONE_TYPE|PRIMARY_NAME_FIRST|PRIMARY_NAME_FULL|PRIMARY_NAME_LAST|PRIMARY_NAME_MIDDLE|PRIMARY_NAME_ORG|RECORD_ID|RECORD_TYPE|SECONDARY_NAME_ORG| SSN_NUMBER|    STATUS|
+-------------+---------

We can see that Robert Smith is in the dataset as Robert, Bob, B, and Robbie, with variations in mailing address and date of birth.

Next, we'll load the `reference` and `watchlist` datasets.

In [5]:
reference = spark.read.json("data/reference.json")

In [6]:
reference.show(10)

+---------------+------------+--------------------+--------------------+----------------+----------+----------+--------------+-----------+----+-------------+-------------+--------------------+--------------------+----------------+----------+------------------+-----------------+-----------------+--------------------+---------+------------+--------------+---------------+----------------+------------------+--------+
|      ADDR_CITY|ADDR_COUNTRY|           ADDR_FULL|          ADDR_LINE1|ADDR_POSTAL_CODE|ADDR_STATE| ADDR_TYPE|      CATEGORY|DATA_SOURCE|DATE|DATE_OF_BIRTH|EMAIL_ADDRESS|       EMPLOYER_NAME|    NATIVE_NAME_FULL|    PHONE_NUMBER|PHONE_TYPE|PRIMARY_NAME_FIRST|PRIMARY_NAME_FULL|PRIMARY_NAME_LAST|    PRIMARY_NAME_ORG|RECORD_ID| RECORD_TYPE|REL_ANCHOR_KEY|REL_POINTER_KEY|REL_POINTER_ROLE|SECONDARY_NAME_ORG|  STATUS|
+---------------+------------+--------------------+--------------------+----------------+----------+----------+--------------+-----------+----+-------------+---------

In [7]:
watchlist = spark.read.json("data/watchlist.json")

In [8]:
watchlist.show(10)

+-------------+------------+--------------------+----------------+----------+---------+--------+-----------+------+-------------+----------------------+---------------------+--------------------+-------------+------+----------------+------------+------------------+-----------------+-----------------+-------------------+---------+-----------+-----------+--------+
|    ADDR_CITY|ADDR_COUNTRY|          ADDR_LINE1|ADDR_POSTAL_CODE|ADDR_STATE|ADDR_TYPE|CATEGORY|DATA_SOURCE|  DATE|DATE_OF_BIRTH|DRIVERS_LICENSE_NUMBER|DRIVERS_LICENSE_STATE|       EMAIL_ADDRESS|EMPLOYER_NAME|GENDER|NATIVE_NAME_FULL|PHONE_NUMBER|PRIMARY_NAME_FIRST|PRIMARY_NAME_FULL|PRIMARY_NAME_LAST|PRIMARY_NAME_MIDDLE|RECORD_ID|RECORD_TYPE| SSN_NUMBER|  STATUS|
+-------------+------------+--------------------+----------------+----------+---------+--------+-----------+------+-------------+----------------------+---------------------+--------------------+-------------+------+----------------+------------+------------------+-----

All three datasets have already been mapped to the [Senzing Entity Specification](https://www.senzing.com/docs/entity_specification/index.html). If you want to use your own data with Senzing, you'll need to map your data to this format.

## Configure Senzing

Next, we need to set everything up so that we can call the Senzing engine on the gRPC server. These steps assume that you're running Senzing locally for development purposes.

Senzing uses an [Abstract Factory](https://garage.senzing.com/sz-sdk-python/senzing.html#module-senzing.szabstractfactory) to create everything that's required to perform entity resolution. We'll create a new abstract factory like so:

In [2]:
grpc_channel = grpc.insecure_channel("localhost:8261")
sz_abstract_factory = SzAbstractFactoryGrpc(grpc_channel)

We'll get the Senzing version details to check connectivity and confirm everything is working.

In [13]:
sz_product = sz_abstract_factory.create_product()
print(json.dumps(json.loads(sz_product.get_version()), indent=2))

{
  "PRODUCT_NAME": "Senzing SDK",
  "VERSION": "4.0.0",
  "BUILD_VERSION": "4.0.0.25136",
  "BUILD_DATE": "2025-05-16",
  "BUILD_NUMBER": "2025_05_16__13_10",
  "COMPATIBILITY_VERSION": {
    "CONFIG_VERSION": "11"
  },
  "SCHEMA_VERSION": {
    "ENGINE_SCHEMA_VERSION": "4.0",
    "MINIMUM_REQUIRED_SCHEMA_VERSION": "4.0",
    "MAXIMUM_REQUIRED_SCHEMA_VERSION": "4.99"
  }
}


Next, we create the Senzing objects that we need: the configuration manager, the diagnostic in case of errors, and the engine that will perform entity resolution when we load our data.

In [14]:
sz_configmanager = sz_abstract_factory.create_configmanager()
sz_diagnostic = sz_abstract_factory.create_diagnostic()
sz_engine = sz_abstract_factory.create_engine()

We create a new config and add the names of the data sources.

In [15]:
config_id = sz_configmanager.get_default_config_id()
sz_config = sz_configmanager.create_config_from_config_id(config_id)

In [16]:
for data_source in ["CUSTOMERS", "REFERENCE", "WATCHLIST"]:
    sz_config.add_data_source(data_source)

And we replace the default config with our updated config:

In [ ]:
new_json_config = sz_config.export()
new_config_id = sz_configmanager.register_config(new_json_config, "Add example data")
sz_configmanager.replace_default_config_id(config_id, new_config_id)

Because we've changed the Senzing configuration, Senzing objects need to be updated. We do this by reinitializing with the new config.

In [18]:
sz_abstract_factory.reinitialize(new_config_id)

## Add records to Senzing

Now that the Senzing engine is set up, we can add our data. We do this using [`sz_engine.add_record()`](https://garage.senzing.com/sz-sdk-python/senzing.html#senzing.szengine.SzEngine.add_record), which adds a single record into the Senzing repository.

This method has three required arguments:
- `data_source_code`, the identifier that we assigned to each dataset when we added the datasets to the Senzing config. In this tutorial, it's one of ['CUSTOMERS', 'REFERENCE', 'WATCHLIST'].
- `record_id`, a unique identifier for each record. This is the `RECORD_ID` column in our example datasets.
- `record_definition`, which is the row (record) we're adding.

So to simply add a record, we would use the following code:

`sz_engine.add_record(record['DATA_SOURCE'],
            record['RECORD_ID'],
            record)`

We'll add an [optional flag](https://senzing.com/docs/4_beta/flags/flags_add/index.html) so that the Senzing engine outputs the entity ID that is affected when we add each row. Then we'll add this entity ID to a Python set (so that no duplicates are possible). We'll use this later on when we want to extract the details of related entities from Senzing.

This is particularly useful when you're adding new data to a large existing dataset, and you only want to see what entities have been affected by the new records.

We'll use Spark's local iterator to iterate through each row in each dataframe, convert each row into a dictionary, and add it to the Senzing repository. We'll extract the entity ID from the info printed out by Senzing:

In [ ]:
def get_affected_entities(info_string):
    # helper function to extract the entity id
    info = json.loads(info_string)
    return [entity["ENTITY_ID"] for entity in info["AFFECTED_ENTITIES"]]

In [20]:
affected_entities = set()

for data_source in [customers, reference, watchlist]:
    for row in data_source.rdd.toLocalIterator():
        record = {k: v for k, v in row.asDict().items() if v is not None}

        info = sz_engine.add_record(
            record["DATA_SOURCE"],
            record["RECORD_ID"],
            record,
            SzEngineFlags.SZ_WITH_INFO,
        )

        affected_entities.update(get_affected_entities(info))
        print(info)

{"DATA_SOURCE":"CUSTOMERS","RECORD_ID":"1001","AFFECTED_ENTITIES":[{"ENTITY_ID":1}]}
{"DATA_SOURCE":"CUSTOMERS","RECORD_ID":"1002","AFFECTED_ENTITIES":[{"ENTITY_ID":1}]}
{"DATA_SOURCE":"CUSTOMERS","RECORD_ID":"1003","AFFECTED_ENTITIES":[{"ENTITY_ID":1}]}
{"DATA_SOURCE":"CUSTOMERS","RECORD_ID":"1004","AFFECTED_ENTITIES":[{"ENTITY_ID":1}]}
{"DATA_SOURCE":"CUSTOMERS","RECORD_ID":"1005","AFFECTED_ENTITIES":[{"ENTITY_ID":1}]}
{"DATA_SOURCE":"CUSTOMERS","RECORD_ID":"1009","AFFECTED_ENTITIES":[{"ENTITY_ID":6}]}
{"DATA_SOURCE":"CUSTOMERS","RECORD_ID":"1010","AFFECTED_ENTITIES":[{"ENTITY_ID":6}]}
{"DATA_SOURCE":"CUSTOMERS","RECORD_ID":"1011","AFFECTED_ENTITIES":[{"ENTITY_ID":8}]}
{"DATA_SOURCE":"CUSTOMERS","RECORD_ID":"1015","AFFECTED_ENTITIES":[{"ENTITY_ID":9}]}
{"DATA_SOURCE":"CUSTOMERS","RECORD_ID":"1016","AFFECTED_ENTITIES":[{"ENTITY_ID":9}]}
{"DATA_SOURCE":"CUSTOMERS","RECORD_ID":"1017","AFFECTED_ENTITIES":[{"ENTITY_ID":9}]}
{"DATA_SOURCE":"CUSTOMERS","RECORD_ID":"1018","AFFECTED_ENTITIES"

When each row is added, we'll see the details printed out. It should look like this for the first row:

`{"DATA_SOURCE":"CUSTOMERS","RECORD_ID":"1001","AFFECTED_ENTITIES":[{"ENTITY_ID":1}]}`

In [21]:
print(affected_entities)

{1, 5, 6, 8, 9, 13, 14, 15, 17, 18, 19, 20, 22, 24, 27, 28, 29, 30, 31, 33, 36, 39, 40, 43, 44, 45, 47, 49, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 76, 77, 79, 81, 83, 85, 87, 89, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 102, 103, 104, 105, 107, 108, 109, 110, 115, 116, 119, 120, 121, 123, 125, 127, 130, 131, 134, 135, 140, 141, 142, 144, 148}


## Process REDO records

The [redo process](https://senzing.zendesk.com/hc/en-us/articles/360007475133-Processing-REDO) in Senzing is a periodic cleanup of the entities in the Senzing repository. You'll need to run the redo process every so often when you're using Senzing with your own data.

The most common use for this is when Senzing discovers a value is overused across entities. You might add 50 records with different names but the same phone number. At first, the shared phone number suggests that these entities are related, but at a certain point the system will spot that the phone number is no longer a good identifier. It will then create redo records in a separate table, and you can run the redo process to clean up the entities.

We'll run the redo process using `sz_engine.get_redo_record()`, which gets each record in the redo table, and `sz_engine.process_redo_record()`, which cleans up the entities based on the redo record. Carrying out this process can generate more records in the redo table, so we'll run it in a `while` loop until there are no more redo records.

We'll also use the `SZ_WITH_INFO` flag again to output the affected entities, and we'll update our set of affected entities with these entity IDs.

In [22]:
while True:
    redo_record = sz_engine.get_redo_record()
    if not redo_record:
        break
    info = sz_engine.process_redo_record(redo_record, flags=SzEngineFlags.SZ_WITH_INFO)
    affected_entities.update(get_affected_entities(info))
    print(info)

{"DATA_SOURCE":"CUSTOMERS","RECORD_ID":"2181","AFFECTED_ENTITIES":[{"ENTITY_ID":102}]}
{"DATA_SOURCE":"CUSTOMERS","RECORD_ID":"2191","AFFECTED_ENTITIES":[{"ENTITY_ID":104}]}
{"DATA_SOURCE":"CUSTOMERS","RECORD_ID":"2192","AFFECTED_ENTITIES":[{"ENTITY_ID":105}]}
{"DATA_SOURCE":"CUSTOMERS","RECORD_ID":"2182","AFFECTED_ENTITIES":[{"ENTITY_ID":103}]}
{"DATA_SOURCE":"CUSTOMERS","RECORD_ID":"2171","AFFECTED_ENTITIES":[{"ENTITY_ID":100},{"ENTITY_ID":101}]}


## Take a look at some results

Let's take a quick look at the entities found by Senzing. We know that there's someone named Robert Smith in the dataset, and we think their date of birth is 11/12/1978. We can look them up in the Senzing repository using `sz_engine.search_by_attributes`.

In [ ]:
search_query = {
    "name_full": "robert smith",
    "date_of_birth": "11/12/1978",
}
search_result = sz_engine.search_by_attributes(json.dumps(search_query))
print(json.dumps(json.loads(search_result), indent=2))

TODO what do we see here?

## Add resolved entities to the Spark dataframe

Our final step in this tutorial is to add a new column to the `customers` dataframe containing a list of all the resolved entities found by Senzing.

So if records 1001, 1002 and 1003 are linked to the same entity, we'll add the list [1002, 1003] to the row for record 1001.

You could then use this information to merge rows in the original dataframe, but we won't cover this in the tutorial.

### Get entity mappings from Senzing


We'll use our list of affected entities to extract all the resolved entities and build a map from the entity ID to all the record IDs for that entity.

In [30]:
def get_records_for_entity(entity_id):
    records = json.loads(sz_engine.get_entity_by_entity_id(entity_id))[
        "RESOLVED_ENTITY"
    ]["RECORDS"]
    return [records[i]["RECORD_ID"] for i in range(len(records))]

In [31]:
# build entity to record map
entity_to_record = {}
for entity in affected_entities:
    try:
        entity_to_record[entity] = get_records_for_entity(entity)
    except SzError:
        entity_to_record[entity] = []

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:8261: Failed to connect to remote host: connect: Connection refused (61)"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:8261: Failed to connect to remote host: connect: Connection refused (61)", grpc_status:14}"
>

## Join entity records to Spark DataFrame

In [25]:
# flatten entity to record map
entity_record_data = [
    (entity_id, record_id)
    for entity_id, records in entity_to_record.items()
    for record_id in records
]
entity_record_df = spark.createDataFrame(entity_record_data, ["ENTITY_ID", "RECORD_ID"])

In [26]:
# group by ENTITY_ID
entity_grouped = entity_record_df.groupBy("ENTITY_ID").agg(
    collect_list("RECORD_ID").alias("ALL_RECORD_IDS")
)

In [27]:
# join to original df
customers = customers.join(entity_record_df, "RECORD_ID", "left")
customers = customers.join(entity_grouped, "ENTITY_ID", "left")

In [28]:
# make new column without original RECORD_ID
customers = customers.withColumn(
    "RELATED_RECORD_IDS",
    array_except(col("ALL_RECORD_IDS"), array(col("RECORD_ID").cast("string"))),
)

In [29]:
customers.show(10, truncate=False)

+---------+---------+-------------+------------+---------+-------------------+----------------+----------+---------+------+--------+-----------+-------+-------------+----------------------+---------------------+------------------+------+-------------------+------------------+----------------+----------------+---------------+------------+----------+------------------+-----------------+-----------------+-------------------+----------------+-----------+------------------+-----------+----------+------------------------------+------------------------+
|ENTITY_ID|RECORD_ID|ADDR_CITY    |ADDR_COUNTRY|ADDR_FULL|ADDR_LINE1         |ADDR_POSTAL_CODE|ADDR_STATE|ADDR_TYPE|AMOUNT|CATEGORY|DATA_SOURCE|DATE   |DATE_OF_BIRTH|DRIVERS_LICENSE_NUMBER|DRIVERS_LICENSE_STATE|EMAIL_ADDRESS     |GENDER|NATIONAL_ID_COUNTRY|NATIONAL_ID_NUMBER|NATIVE_NAME_FULL|PASSPORT_COUNTRY|PASSPORT_NUMBER|PHONE_NUMBER|PHONE_TYPE|PRIMARY_NAME_FIRST|PRIMARY_NAME_FULL|PRIMARY_NAME_LAST|PRIMARY_NAME_MIDDLE|PRIMARY_NAME_ORG|RECOR

25/07/08 12:52:45 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 912288 ms exceeds timeout 120000 ms
25/07/08 12:52:45 WARN SparkContext: Killing executors is not supported by current scheduler.
25/07/08 12:52:46 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:81)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:669)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1296)
	at o

In [24]:
spark.stop()